<a href="https://colab.research.google.com/github/seokhee516/ML-DL-playground/blob/main/h1n1_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -U keras-tuner

     |████████████████████████████████| 98 kB 4.1 MB/s 


In [7]:
! pip install category_encoders

     |████████████████████████████████| 82 kB 448 kB/s 


In [36]:
import numpy as np
from sklearn.impute import SimpleImputer

In [8]:
import pandas as pd

from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras
import tensorflow as tf
import IPython
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import kerastuner as kt
from tensorflow.keras import regularizers

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  del sys.path[0]


In [44]:
# 데이터 불러오기 
target = 'vacc_h1n1_f'
train = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

In [45]:
train, val = train_test_split(train, test_size=len(test), stratify=train[target], random_state=2)
# def engineer(df):    
#     #df.loc[(df['rent_own_r']!= 1.0) & (df['rent_own_r']!= 2.0),'rent_own_r'] = 2.0

#     behaviorals = [col for col in df.columns if 'behavioral' in col] 
#     df['behaviorals'] = df[behaviorals].sum(axis=1)
    
#     fixed_data = []
#     for i in df["employment_status"]:
#       if i == "Not in Labor Force":
#         fixed_data.append("Unemployed")
#       else:
#         fixed_data.append(i)
#     df["employment_status"] = fixed_data
    
#     dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
#     df.drop(columns=dels, inplace=True)
#     df.dropna(thresh=int(len(df) * 0.5), axis=1, inplace=True)
        
#     return df


# train = engineer(train)
# val = engineer(val)
# test = engineer(test)

features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [81]:
encoder = OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

simp = SimpleImputer(strategy='mean')
X_train_inputed= simp.fit_transform(X_train_encoded)
X_val_inputed = simp.fit_transform(X_val_encoded)
X_test_inputed = simp.fit_transform(X_test_encoded)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_inputed)
X_val_scaled = scaler.transform(X_val_inputed)
X_test_scaled = scaler.transform(X_test_inputed)

In [48]:
# 모델 만들기

def model_builder(hp):

  model = Sequential()

  # Dense layer에서 노드 수를 조정(32-512)
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)

  model.add(Dense(units = hp_units, activation='relu'))
  model.add(Dense(units = hp_units, activation='relu'))

  model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

  # Optimizer의 학습률(learning rate)을 조정[0.01, 0.001, 0.0001]합니다. 
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  # 컴파일 단계, 옵티마이저와 손실함수, 측정지표를 연결해서 계산 그래프를 구성함
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = hp_learning_rate), 
                loss=tf.keras.losses.BinaryCrossentropy(from_logits = True), 
                metrics=['accuracy'])

  return model

In [72]:
# 튜너를 인스턴스화하고 하이퍼 튜닝을 수행
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 30, 
                     factor = 3,
                     directory = 'my_dir3',
                     project_name = 'intro_to_kt')

In [54]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [73]:
tuner.search(X_train_inputed, y_train, epochs = 30, batch_size=50, validation_data = (X_val_inputed,y_val), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
최적화된 Dense 노드 수 : {best_hps.get('units')} 
최적화된 Learning Rate : {best_hps.get('learning_rate')} 
""")

Trial 9 Complete [00h 00m 03s]
val_accuracy: 0.7814546227455139

Best val_accuracy So Far: 0.8006333708763123
Total elapsed time: 00h 00m 44s

Search: Running Trial #10

Hyperparameter    |Value             |Best Value So Far 
units             |512               |480               
learning_rate     |0.001             |0.01              
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |3                 |3                 
tuner/round       |0                 |0                 

Epoch 1/2
281/281 [==============================] - 2s 6ms/step - loss: 0.5370 - accuracy: 0.7660 - val_loss: 0.4682 - val_accuracy: 0.7985
Epoch 2/2
 24/281 [=>............................] - ETA: 0s - loss: 0.4877 - accuracy: 0.7850

KeyboardInterrupt: ignored

In [56]:
tf.random.set_seed(1442)
initializer = tf.keras.initializers.HeNormal()

model = Sequential()

model.add(Dense(best_hps.get('units'), 
                activation='relu', kernel_initializer=initializer,          
                kernel_regularizer=regularizers.l2(0.01),    # L2 norm regularization
                activity_regularizer=regularizers.l1(0.01))) # L1 norm regularization))
model.add(Dense(best_hps.get('units'),
                activation='relu', kernel_initializer=initializer,            
                kernel_regularizer=regularizers.l2(0.01),    # L2 norm regularization
                activity_regularizer=regularizers.l1(0.01)))
model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = best_hps.get('learning_rate')), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

results = model.fit(X_train_inputed, y_train, epochs=2, batch_size=50, validation_data=(X_val_inputed,y_val))

Epoch 1/2
281/281 [==============================] - 2s 6ms/step - loss: 1.3849 - accuracy: 0.7597 - val_loss: 0.4992 - val_accuracy: 0.7610
Epoch 2/2
281/281 [==============================] - 1s 5ms/step - loss: 0.4954 - accuracy: 0.7858 - val_loss: 0.4888 - val_accuracy: 0.7968


In [97]:
# 테스트셋 사용해서 결과 보기
model.evaluate(X_val_scaled,  y_val, verbose=2)

879/879 - 1s - loss: 0.4888 - accuracy: 0.7968 - 1s/epoch - 2ms/step


[0.488782674074173, 0.7967904806137085]

In [83]:
from sklearn.tree import DecisionTreeClassifier
test_model = DecisionTreeClassifier(max_depth=10)

test_model.fit(X_train_scaled, y_train)
test_y_pred = model.predict(X_val_scaled)
test_y_pred

array([[0.3413249 ],
       [0.53947824],
       [0.53947824],
       ...,
       [0.12606831],
       [0.11883104],
       [0.20996739]], dtype=float32)

In [107]:
from sklearn.metrics import classification_report
print(classification_report(y_val, np.where(test_y_pred[:,0]>0.5, 1, 0)))

              precision    recall  f1-score   support

           0       0.82      0.94      0.88     21386
           1       0.64      0.34      0.45      6718

    accuracy                           0.80     28104
   macro avg       0.73      0.64      0.66     28104
weighted avg       0.78      0.80      0.77     28104



In [58]:
predictions = model.predict(X_test_scaled)

In [66]:
predictions

array([[0.07474314],
       [0.14654897],
       [0.13029197],
       ...,
       [0.11883104],
       [0.11883104],
       [0.49666956]], dtype=float32)

In [111]:
 y_pred_test = np.where(predictions[:,0]>0.5, 1, 0).astype(int)

In [112]:
id = pd.Series(range(len(y_pred_test)))

In [113]:
y_pred_test = pd.Series(y_pred_test)

In [117]:
y_pred_test.value_counts()

0    24540
1     3564
dtype: int64

In [115]:
submission = pd.concat([id, y_pred_test], axis=1)
submission

,0,1
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
28099,28099,0
28100,28100,0
28101,28101,0
28102,28102,0


In [118]:
submission.rename(columns={0:"id", 1:target}, inplace=True)
submission.to_csv("deep_submission.csv",index=False)